# FYSS5120 Efficient Numerical Programming - Demo 2

Author: Felix Cahyadi

Date: 19.09.2023

## 1. Time NumPy sum() method vs. Python sum() function

In [2]:
# import time to measure time
from time import perf_counter


In [3]:
# prepare the array that we want to sum over
import numpy as np
# Initialize np.random seed
np.random.seed(20092023)


N = 10000000
A = np.random.random(N)

In [4]:
# timing the NumPy sum() method
tic = perf_counter()
sum_np = np.sum(A)
toc = perf_counter()
print("The value of the sum:",sum_np)
print("The time needed to sum the array using NumPy is:", toc-tic,"seconds")

The value of the sum: 4999440.177048801
The time needed to sum the array using NumPy is: 0.02069660008419305 seconds


In [5]:
# timing the Python sum() function
tic = perf_counter()
sum_bi = sum(A)
toc = perf_counter()
print("The value of the sum:",sum_bi)
print("The time needed to sum the array using Python built-in function is:", toc-tic, "seconds")

The value of the sum: 4999440.177049306
The time needed to sum the array using Python built-in function is: 0.7354535999475047 seconds


### Conclusion: The Python built-in function takes more time to sum the values compared to the NumPy function.

## Write a Python code that computes the distances of 1000 particles in three-dimensional space. The particle coordinates are in the NumPy NxD array x.

### Time the five ways to compute particle distances using, for example, time.perf_counter():

In [6]:
# Initialize the array, the dimension of the array is N x D, where N is the number of particles and D is the number of coordinates describing the position of the particle

N = 1000
D = 3
coor_arr = np.random.random((N,D))
print("This is the coordinate array:", coor_arr)
print("The shape of the NxD array:",coor_arr.shape)

This is the coordinate array: [[0.16903913 0.38412363 0.40311881]
 [0.27801375 0.16623097 0.67475415]
 [0.25267802 0.53406194 0.31159744]
 ...
 [0.26628117 0.81110868 0.1184048 ]
 [0.49298813 0.9692693  0.36463858]
 [0.49425334 0.2028975  0.99916542]]
The shape of the NxD array: (1000, 3)


In [7]:
# Checking the value of the distance between the 1st particle and 2nd particle
np.sqrt((0.16903913-0.27801375)**2 + (0.38412363-0.16623097)**2 + (0.40311881-0.67475415)**2)

0.3648814013113516

(a) NumPy broadcasting used in sample code potential_simple.py

In [8]:
def distance_broadcast(x):
    d = x[:,np.newaxis,:]-x
    r = np.sqrt((d**2).sum(2))
    rs = r[np.triu_indices_from(r,1)]
    return rs

In [9]:
tic = perf_counter()
dist_broad = distance_broadcast(coor_arr)
toc = perf_counter()
print('The matrix shape is:',dist_broad.shape)
print("First 10 result of the sum:",dist_broad[0:10])
print("Time needed for this method is:",toc-tic,"second")

The matrix shape is: (499500,)
First 10 result of the sum: [0.3648814  0.19455879 0.61243924 0.58275348 0.64917966 0.37143821
 0.7321392  0.88522493 0.59029583 0.79632249]
Time needed for this method is: 0.11298199999146163 second


(b) A version that uses numpy.linalg.norm() to compute r in the previous code.

In [10]:
# Import library
from numpy.linalg import norm

In [11]:
def distance_norm(arr):
    d = arr[:,np.newaxis,:]-arr
    r = norm(d, ord=2, axis=2) # sum over the d axis
    rs = r[np.triu_indices_from(r,1)]
    return rs

In [12]:
tic = perf_counter()
dist_norm = distance_norm(coor_arr)
toc = perf_counter()
print('The matrix shape is:', dist_norm.shape)
print("First 10 result of the sum:",dist_norm[0:10])
print("Time needed for this method is:",toc-tic,"second")

The matrix shape is: (499500,)
First 10 result of the sum: [0.3648814  0.19455879 0.61243924 0.58275348 0.64917966 0.37143821
 0.7321392  0.88522493 0.59029583 0.79632249]
Time needed for this method is: 0.09493960009422153 second


(c) Compute r using numpy.einsum()

In [13]:
def distance_einstein(arr):
    d = arr[:,np.newaxis,:] - arr
    r = np.sqrt(np.einsum('ijk,ijk->ij',d,d)) # Using Einstein notation
    rs = r[np.triu_indices_from(r,1)]
    return rs

In [14]:
tic = perf_counter()
dist_einsum = distance_einstein(coor_arr)
toc = perf_counter()
print('The matrix shape is:', dist_einsum.shape)
print("First 10 result of the sum:",dist_einsum[0:10])
print("Time needed for this method is:",toc-tic,"second")

The matrix shape is: (499500,)
First 10 result of the sum: [0.3648814  0.19455879 0.61243924 0.58275348 0.64917966 0.37143821
 0.7321392  0.88522493 0.59029583 0.79632249]
Time needed for this method is: 0.05794550001155585 second


(d) A version that uses scipy.spatial.distance.pdist.

In [15]:
from scipy.spatial.distance import pdist

In [16]:
def distance_pdist(arr):
    rs = pdist(arr)
    return rs

In [17]:
tic = perf_counter()
dist_pdist = distance_pdist(coor_arr)
toc = perf_counter()
print('The matrix shape is:', dist_pdist.shape)
print("First 10 result of the sum:",dist_pdist[0:10])
print("Time needed for this method is:",toc-tic,"second")

The matrix shape is: (499500,)
First 10 result of the sum: [0.3648814  0.19455879 0.61243924 0.58275348 0.64917966 0.37143821
 0.7321392  0.88522493 0.59029583 0.79632249]
Time needed for this method is: 0.006051399977877736 second


(e) A basic for-loop, but with Numba @njit decorator

In [18]:
# import library
from numba import njit

In [19]:
@njit(fastmath = True)
def distance_njit(arr):
    d = arr[:,np.newaxis,:] - arr

    # start the loop
    ni,nj,nk = d.shape
    r = np.zeros((ni,nj))
    for i in range(ni):
        for j in range(nj):
            sum_sq = 0.0
            for k in range(nk):
                sum_sq += d[i,j,k]**2
            r[i,j] = np.sqrt(sum_sq)

    rs = []
    for i in range(ni):
        for j in range(i+1,nj):
            rs.append(r[i,j])
    
    rs = np.array(rs)

    return rs

In [20]:
tic = perf_counter()
dist_njit = distance_njit(coor_arr)
toc = perf_counter()
print('The matrix shape is:',dist_njit.shape)
print("First 10 result of the sum:",dist_njit[0:10])
print("Time needed for this method is:",toc-tic,"second")

The matrix shape is: (499500,)
First 10 result of the sum: [0.3648814  0.19455879 0.61243924 0.58275348 0.64917966 0.37143821
 0.7321392  0.88522493 0.59029583 0.79632249]
Time needed for this method is: 1.994969200110063 second


In [21]:
# do some checking, assume that the 1st method gave us the correct answer

# compare 1st and 2nd method
print("difference between 1st and 2nd method:",norm(dist_broad-dist_norm))

# compare 1st and 3rd method
print("difference between 1st and 3rd method:",norm(dist_broad-dist_einsum))

# compare 1st and 4th method
print("difference between 1st and 4th method:",norm(dist_broad-dist_pdist))

# compare 1st and 5th method
print("difference between 1st and 5th method:",norm(dist_broad-dist_njit))


difference between 1st and 2nd method: 0.0
difference between 1st and 3rd method: 2.6961259399820837e-14
difference between 1st and 4th method: 0.0
difference between 1st and 5th method: 2.5592297614755837e-14


### Conclusion: We can say that our implementation is correct, because all of them gave us the same result. The fastest method is the pdist function, while the basic for-loop with the Numba decorator is the slowest.